# Resident Population

In [ ]:
import pandas as pd

states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",
    "District of Columbia", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa",
    "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota",
    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", 
    "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", 
    "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah",
    "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"
]

pop_list = []
for state in states:
    file = f"../input_data/Resident Population in {state}.csv"
    df = pd.read_csv(file)
    df = df.rename(columns={df.columns[0]: 'year', df.columns[1]: 'population'})
    df['state'] = state
    pop_list.append(df)
df_pop = pd.concat(pop_list, ignore_index=True)
def extract_year(val):
    try:
        return int(val)
    except:
        return int(str(val)[:4])

df_pop['year'] = df_pop['year'].apply(extract_year)


# Stack Real Median Household Income

In [ ]:
income_list = []
for state in states:
    file = f"../input_data/Real Median Household Income in {state}.csv"
    df = pd.read_csv(file)
    df = df.rename(columns={df.columns[0]: 'year', df.columns[1]: 'median_income'})
    df['state'] = state
    income_list.append(df)
df_income = pd.concat(income_list, ignore_index=True)
df_pop['year'] = df_pop['year'].apply(extract_year)
df_income['year'] = df_income['year'].apply(extract_year)

df_annual = pd.merge(df_pop, df_income, on=['state', 'year'], how='outer')

# Expand Annual Data to Monthly

In [ ]:
min_year, max_year = df_annual['year'].min(), df_annual['year'].max()
months = pd.date_range(f'{min_year}-01-01', f'{max_year}-12-01', freq='MS')
all_state_months = pd.MultiIndex.from_product([states, months], names=['state', 'month']).to_frame(index=False)
all_state_months['year'] = all_state_months['month'].dt.year

# Merge annual data into monthly grid
df_monthly = pd.merge(all_state_months, df_annual, on=['state', 'year'], how='left')
df_monthly = df_monthly.sort_values(['state', 'month'])
df_monthly[['population', 'median_income']] = (
    df_monthly.groupby('state')[['population', 'median_income']].fillna(method='ffill')
)

In [ ]:
df_monthly['month'] = pd.to_datetime(df_monthly['month'])

start_date = '1987-01-01'
end_date = '2024-12-01'

df_monthly = df_monthly[(df_monthly['month'] >= start_date) & (df_monthly['month'] <= end_date)]

# Unemployment Rate

In [ ]:
df_unemp = pd.read_csv("../input_data/Unemployment Rate_1976.csv")
df_unemp['year'] = pd.to_datetime(df_unemp['Year'].astype(str) + '-' + df_unemp['Month'].astype(str).str.zfill(2) + '-01')
df_unemp = df_unemp[(df_unemp['year'] >= '1987-01-01') & (df_unemp['year'] <= '2024-12-01')]
df_unemp = df_unemp.rename(columns={
    'State': 'state',
    'Unemployment Rate': 'unemployment_rate'
})
df_unemp = df_unemp[['state', 'year', 'unemployment_rate']]

In [ ]:
df_unemp = df_unemp.reset_index(drop=True)
df_unemp = df_unemp.rename(columns={'year': 'month'})
df_final = pd.merge(df_monthly, df_unemp, on=['state', 'month'], how='left')

# Initial Claims

In [ ]:
files = [
    "../input_data/Initial Claims_Alabama_to_Iowa.csv",
    "../input_data/Initial Claims_Kansas_to_Oklahoma.csv",
    "../input_data/Initial Claims_Oregan_to_PuertoRico.csv"
]
claims_list = [pd.read_csv(file) for file in files]
df_claims = pd.concat(claims_list, ignore_index=True)

df_claims = df_claims.rename(columns={
    'State': 'state',
    'Filed week ended': 'filed_week_ended',
    'Initial Claims': 'initial_claims'
})

df_claims['filed_week_ended'] = pd.to_datetime(df_claims['filed_week_ended'])

In [ ]:
df_claims.isnull().sum()

In [ ]:
df_claims['initial_claims'] = df_claims['initial_claims'].replace(',', '', regex=True).astype(int)

df_claims['month'] = df_claims['filed_week_ended'].values.astype('datetime64[M]')
df_claims_monthly = (
    df_claims.groupby(['state', 'month'], as_index=False)['initial_claims'].sum()
)
df_final = pd.merge(df_final, df_claims_monthly, on=['state', 'month'], how='left')

In [ ]:
df_final.head()